In [11]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
navertoon = pd.read_csv("NaverToonInfo_tag.csv")
navertoon = navertoon.fillna('성인웹툰')
navertoon['total'] = navertoon['스토리작가']+','+navertoon['형식']+','+navertoon['장르']+','+navertoon['이용가']+','+navertoon['태그']

In [13]:
navertoon_df = navertoon[['제목','평점','스토리작가','형식',
                          '장르','이용가','조회순','total']]

navertoon_df['total'][0]

'이동건,에피소드,로맨스,12세 이용가,드라마&영화 원작웹툰,로맨스,완결로맨스'

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

def find_sim_toon_ver1(navertoon_df, sorted_ind, title_name, top_n=10):
    
    title_toon = navertoon_df[navertoon_df['제목'] == title_name]
    
    title_index = title_toon.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)    
    
    similar_indexes = similar_indexes.reshape(-1)

    return navertoon_df.iloc[similar_indexes].sort_values('조회순',ascending=True)[:10]

In [15]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(navertoon_df['total'])


genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:10])

(1383, 1383)
[[1.         0.25031309 0.14285714 ... 0.36313652 0.         0.12104551]
 [0.25031309 1.         0.05006262 ... 0.57265629 0.12725695 0.12725695]
 [0.14285714 0.05006262 1.         ... 0.06052275 0.06052275 0.06052275]
 ...
 [0.05634362 0.05923489 0.11268723 ... 0.07161149 0.07161149 0.07161149]
 [0.12104551 0.12725695 0.06052275 ... 0.15384615 0.15384615 0.84615385]
 [0.22537447 0.05923489 0.11268723 ... 0.07161149 0.07161149 0.5728919 ]]


In [16]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:5])

[[   0  321  623 ... 1264 1265  395]
 [   1  131  249 ...  154  855  190]
 [   2   65  292 ...  240 1158  898]
 [   3  691  353 ...  361  765  135]
 [   4  882  543 ...  130 1250 1035]]


In [17]:
title_toon = navertoon_df[navertoon_df['제목'] == '신의 탑']
title_index = title_toon.index.values
similar_indexes = genre_sim_sorted_ind[title_index, :10]


print(similar_indexes)

[[   2   65  292  149   20    3 1013  831  790 1152]]


In [18]:
similar_toon = find_sim_toon_ver1(navertoon_df, genre_sim_sorted_ind, '신의 탑', 10)
similar_toon[['제목', '평점', '조회순', 'total']]

[[   2   65  292  149   20    3 1013  831  790 1152]]


,제목,평점,조회순,total
2,신의 탑,9.89,3,"SIU,스토리,판타지,12세 이용가,세계관,이능력배틀물,소년왕도물,먼치킨,판타지"
3,갓 오브 하이스쿨,9.64,4,"박용제,스토리,판타지,12세 이용가,이능력배틀물,먼치킨,판타지,완결무료,완결판타지"
20,노블레스,9.62,21,"손제호,스토리,판타지,12세 이용가,먼치킨,판타지,완결판타지"
65,나노리스트,9.98,66,"민송아,스토리,판타지,12세 이용가,먼치킨,판타지,완결판타지"
149,레사 시즌2~3,9.97,150,"POGO,스토리,판타지,12세 이용가,먼치킨,판타지,완결판타지"
292,레사 시즌1,9.95,293,"POGO,스토리,판타지,12세 이용가,먼치킨,판타지,완결판타지"
790,아이레,9.83,791,"설이,스토리,판타지,12세 이용가,판타지"
831,블랙시저스,9.95,832,"유령,스토리,판타지,12세 이용가,판타지"
1013,길티액스,9.57,1014,"정샛별,스토리,판타지,12세 이용가,판타지"
1152,멸망X초이스,9.83,1153,"텐,스토리,판타지,12세 이용가,판타지,완결판타지"
